<a href="https://colab.research.google.com/github/MrCarry123/OrganSegmentation/blob/main/testing_segformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install numpy==1.19.5
!pip3 install tensorflow==2.13.0
!pip3 install keras==2.13.1
!pip3 install -U segmentation-models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import tensorflow as tf
print(tf.config.experimental.list_physical_devices('GPU'))
import tensorflow.keras
drive.mount('/content/drive')
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
tf.keras.backend.set_image_data_format('channels_last')

[]
Mounted at /content/drive
env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Organsegmentation2/')

In [ ]:
import tensorflow as tf
# import tensorflow.keras
import segmentation_models as sm
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
# from tensorflow.keras.utils import to_categorical
import os
import time
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from sklearn.metrics import precision_score, recall_score
from glob import glob
from skimage.transform import resize
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, concatenate, BatchNormalization, Activation, MaxPool2D
import numpy as np
from tqdm import tqdm
from PIL import Image
import random
import matplotlib.pyplot as plt

In [ ]:
from modules import MixVisionTransformer
from Head import SegFormerHead
from utils import ResizeLayer

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

gpu_number = -1 #set gpu number here
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[gpu_number], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

In [ ]:
# %env SM_FRAMEWORK=tf.keras
# tf.keras.backend.set_image_data_format('channels_last')

In [ ]:
test_images = glob('/content/drive/MyDrive/Organsegmentation2/test_images/*.png')[:2000]
test_labels = glob('/content/drive/MyDrive/Organsegmentation2/test_labels/*.png')[:2000]
len(test_labels)

2000

In [ ]:
"""def load_test_data(test_images, test_labels):
  images = []
  labels = []
  for i in tqdm(range(len(test_images_names))):
    x = Image.open(test_images+test_images_names[i])
    x = np.asarray(x, dtype=np.float32)
    # print(file_path, type(file_path))
    label = Image.open(test_labels+test_labels_names[i])
    label = np.asarray(label, dtype=np.float32)
    resized_images = resize(x, (32,32), mode='constant', anti_aliasing=True)

    # Resize labels to 128x128x4
    resized_labels = resize(label, (32,32), mode='constant', anti_aliasing=True)
    resized_labels[resized_labels != 0] = 1
    images.append(resized_images)
    labels.append(resized_labels)
  return images, labels """

"def load_test_data(test_images, test_labels):\n  images = []\n  labels = []\n  for i in tqdm(range(len(test_images_names))):\n    x = Image.open(test_images+test_images_names[i])\n    x = np.asarray(x, dtype=np.float32)\n    # print(file_path, type(file_path))\n    label = Image.open(test_labels+test_labels_names[i])\n    label = np.asarray(label, dtype=np.float32)\n    resized_images = resize(x, (32,32), mode='constant', anti_aliasing=True)\n\n    # Resize labels to 128x128x4\n    resized_labels = resize(label, (32,32), mode='constant', anti_aliasing=True)\n    resized_labels[resized_labels != 0] = 1\n    images.append(resized_images)\n    labels.append(resized_labels)\n  return images, labels "

In [ ]:
from tqdm import tqdm
def load_test_data(test_images, test_labels):
  images = []
  labels = []
  test_images_names = sorted(test_images)
  test_labels_names = sorted(test_labels)
  for i in tqdm(range(len(test_images_names))):
    x = Image.open(test_images_names[i])
    x = np.asarray(x, dtype=np.float32)
    # print(file_path, type(file_path))
    label = Image.open(test_labels_names[i])
    label = np.asarray(label, dtype=np.float32)
    resized_images = resize(x, (32,32), mode='constant', anti_aliasing=True)
    resized_images = resized_images[:, :, :3]
    # Resize labels to 128x128x4
    resized_labels = resize(label, (32,32), mode='constant', anti_aliasing=True)
    resized_labels = resized_labels [:, :, :3]
    resized_labels[resized_labels != 0] = 1
    images.append(resized_images)
    labels.append(resized_labels)
  return images, labels

In [ ]:
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=True)

In [ ]:
def DiceBCELoss(targets, inputs):
    smooth = 1e-6
    inputs = tf.keras.backend.flatten(inputs)
    targets = tf.keras.backend.flatten(targets)

    BCE = tf.keras.losses.binary_crossentropy(targets, inputs)
    intersection = tf.reduce_sum(targets * inputs)
    dice_loss = 1.0 - (2.0 * intersection + smooth) / (tf.reduce_sum(targets) + tf.reduce_sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    return Dice_BCE

In [ ]:
saved_model_path = '/content/drive/MyDrive/Organsegmentation2/Segformer.h5'

In [ ]:
# custom_object = {'MixVisionTransformer': MixVisionTransformer}
custom_object = {
    'MixVisionTransformer': MixVisionTransformer,
    'SegFormerHead': SegFormerHead,
    'ResizeLayer': ResizeLayer
    # Add more custom layers as needed
}

In [ ]:
model = tf.keras.models.load_model(
    saved_model_path, custom_objects=custom_object, compile=False)

In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 mix_vision_transformer_1 (  [(None, 8, 8, 64),           4407212   ['input_1[0][0]']             
 MixVisionTransformer)        (None, 4, 4, 128),          8                                       
                              (None, 2, 2, 320),                                                  
                              (None, 1, 1, 512)]                                                  
                                                                                                  
 seg_former_head_1 (SegForm  (None, 8, 8, 3)              3154179   ['mix_vision_transformer_1

In [ ]:
model.compile(loss=DiceBCELoss, optimizer=opt,metrics=[sm.metrics.Precision(), sm.metrics.Recall(), sm.metrics.IOUScore()])
images1, labels1 = load_test_data(test_images,test_labels)

100%|██████████| 2000/2000 [02:56<00:00, 11.33it/s]


In [ ]:
images1 = np.array(images1)
labels1= np.array(labels1)

In [ ]:
images1.shape, labels1.shape

In [ ]:
# model.evaluate(tf.convert_to_tensor(images1,dtype=tf.float32),tf.convert_to_tensor(labels1, dtype=tf.float32))

In [ ]:
# labels1

In [ ]:
# loss, precision, recall, iou = model.evaluate(images1, labels1)

In [ ]:
results = model.evaluate(images1, labels1)

204/204 [==============================] - 263s 1s/step - loss: 1.3876 - precision: 0.5582 - recall: 0.3333 - iou_score: 0.2556


In [ ]:
# !jupyter nbconvert --to  script 'testing.ipynb'

In [ ]:
loss = results[0]
precision = results[1]
recall = results[2]
iou_score = results[3]

In [ ]:
print('loss:',loss, 'precision:',precision, 'recall:',recall, 'iou_score:',iou_score)

In [ ]:
predicted_labels = model.predict(images1)

In [ ]:
slice_index = 2  # Choose the slice index you want to visualize
file_index = 20

plt.figure(figsize=(12, 5))

# Display the original image
plt.subplot(1, 3, 1)
plt.imshow(images1[file_index][:, :, slice_index], cmap='jet')
plt.title('original image')
plt.axis('off')

# Display the original label
plt.subplot(1, 3, 2)
plt.imshow(labels1[file_index][:, :, slice_index], cmap='jet')
plt.title('original label')
plt.axis('off')

# Display the predicted label
plt.subplot(1, 3, 3)
plt.imshow(predicted_labels[file_index][:, :, slice_index], cmap='jet')
plt.title('predicted Label')
plt.axis('off')

plt.tight_layout()
plt.show()
